In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("../../data/df_segmentation_P.csv", index_col=[0])
df["date_mensuelle"] = pd.to_datetime(df["date_mensuelle"])
df['date_trimestrielle'] = df['date_mensuelle'].dt.year.astype(str) + '_' + df['date_mensuelle'].dt.quarter.astype(str)

# MOC C

In [5]:
resultats = df.groupby("Classes").agg(moyenne_TARGET=("TARGET", "mean")).to_dict()["moyenne_TARGET"]

In [6]:
resultats

{1: 0.25674217907227614,
 2: 0.17383829218205263,
 3: 0.12256764090239554,
 4: 0.07622553278882763,
 5: 0.050153307052124396,
 6: 0.03261193020424726,
 7: 0.02263618402483771}

In [7]:
def Boostrapping_Classes(classe):
    df_classe = df[df['Classes'] == classe]
    tx_defaut_liste = []

    for _ in range(1000):
        echantillon = df_classe["TARGET"].sample(n=len(df_classe), replace=True)
        tx_defaut_liste.append(echantillon.mean())
        
    tx_defaut_liste = np.array(tx_defaut_liste)

    decile9 = np.percentile(tx_defaut_liste, 90)

    MOC_C = decile9 - resultats[classe]
    
    return(classe, resultats[classe], decile9, MOC_C)

In [8]:
MOC_C = pd.DataFrame(columns = ["Classe", "LRA", "Décile9", "Moc_C"])

In [9]:
for i in range(1,8):
    classe, LRA, decile9, MOC_C_classe = Boostrapping_Classes(i)
    MOC_C.loc[len(MOC_C)] = [classe, LRA, decile9, MOC_C_classe]

In [10]:
MOC_C

,Classe,LRA,Décile9,Moc_C
0,1.0,0.256742,0.261784,0.005041
1,2.0,0.173838,0.177277,0.003439
2,3.0,0.122568,0.124609,0.002042
3,4.0,0.076226,0.077491,0.001265
4,5.0,0.050153,0.051147,0.000993
5,6.0,0.032612,0.033464,0.000852
6,7.0,0.022636,0.024047,0.001411


# MOC A
### Crise Covid

In [11]:
avant_covid = df[df['date_trimestrielle'] < '2020_2']

taux_defaut_avant_covid = avant_covid.groupby('Classes')['TARGET'].mean()
taux_defaut_pendant_covid = df.groupby('Classes')['TARGET'].mean()

impact_covid_par_classe = (taux_defaut_avant_covid/taux_defaut_pendant_covid)-1

In [12]:
impact_covid_par_classe

Classes
1    0.000614
2   -0.002937
3   -0.007154
4   -0.003364
5   -0.009690
6   -0.002816
7   -0.013034
Name: TARGET, dtype: float64

In [13]:
variance_taux_defaut_pendant_covid = df.groupby('Classes')['TARGET'].var()
variance_taux_defaut_pendant_covid

Classes
1    0.190839
2    0.143625
3    0.107548
4    0.070416
5    0.047639
6    0.031549
7    0.022125
Name: TARGET, dtype: float64

In [14]:
simulations = pd.DataFrame()

for classe in impact_covid_par_classe.index:
    moyenne_impact = impact_covid_par_classe[classe]
    variance_impact = variance_taux_defaut_pendant_covid[classe]

    variance_impact = variance_impact if variance_impact > 0 else 0.001

    tirages = np.random.normal(moyenne_impact, variance_impact, 1000)

    simulations[classe] = tirages
    
simulations

,1,2,3,4,5,6,7
0,0.046132,0.019495,0.150000,-0.153623,-0.064094,-0.009338,0.012030
1,0.073875,0.270020,0.106451,0.125480,-0.131534,0.044782,-0.026733
2,-0.265271,-0.143513,-0.014463,-0.092259,-0.035069,-0.006863,-0.005180
3,-0.072478,0.020564,-0.085758,0.102765,0.058380,0.010071,-0.005464
4,0.004406,-0.084484,0.095201,-0.000529,-0.076346,0.011909,-0.027805
...,...,...,...,...,...,...,...
995,-0.218146,-0.211661,-0.008148,-0.199096,-0.072964,0.054705,-0.011457
996,0.068108,0.190884,-0.041770,-0.124589,-0.087080,-0.046353,-0.022066
997,-0.293067,-0.256781,0.168177,-0.109017,-0.077985,0.031303,-0.052808
998,0.392361,-0.268687,-0.059230,0.020920,-0.015399,-0.027881,-0.016212


In [15]:
moyenne_empirique = simulations.mean()
percentile_90 = simulations.quantile(0.90)
MOC_A = percentile_90 - taux_defaut_pendant_covid
MOC_A

1   -0.007236
2    0.002676
3    0.001133
4    0.012848
5    0.000612
6    0.006496
7   -0.006731
dtype: float64

# Conclusion

In [16]:
MOC = pd.DataFrame(columns = ["Classe", "LRA", "MOC_A", "MOC_C"])
MOC["LRA"] = taux_defaut_avant_covid.reset_index(drop=True)
MOC["MOC_A"] = MOC_A.reset_index(drop=True).apply(lambda x : 0 if x < 0 else x)
MOC["MOC_C"] = MOC_C["Moc_C"].reset_index(drop=True)
MOC["Classe"] = range(1,8)
MOC["LRA + MOC"] = MOC["LRA"] + MOC["MOC_A"] + MOC["MOC_C"]

In [17]:
MOC

,Classe,LRA,MOC_A,MOC_C,LRA + MOC
0,1,0.256900,0.000000,0.005041,0.261941
1,2,0.173328,0.002676,0.003439,0.179442
2,3,0.121691,0.001133,0.002042,0.124865
3,4,0.075969,0.012848,0.001265,0.090083
4,5,0.049667,0.000612,0.000993,0.051273
5,6,0.032520,0.006496,0.000852,0.039869
6,7,0.022341,0.000000,0.001411,0.023752
